In this notebook, you should implement a first version of a working machine learning model to predict the age of an Abalone.

A few guidelines:
- The model does not have to be complex. A simple linear regression model is enough.
- You should use MLflow to track your experiments. You can use the MLflow UI to compare your experiments.
- Do not push any MLflow data to the repository. Only the code to run the experiments is interesting and should be pushed.

## Imports

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import warnings
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import r2_score
import mlflow
from mlflow.tracking import MlflowClient

In [ ]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

In [ ]:
def compute_metrics(y, y_pred):
    rmse = mean_squared_error(y, y_pred, squared=False)
    mape = mean_absolute_percentage_error(y, y_pred)

    # Log metrics
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mape", mape)

    print(rmse, mape)

In [ ]:
# Set the experiment name
mlflow.set_experiment("abalone-experiment-2")

# Start a run
with mlflow.start_run() as run:
    run_id = run.info.run_id

    # Set tags for the run
    mlflow.set_tag("project", "abalone")
    mlflow.set_tag("team", "team1")

    # Load data (REPLACE PATH)
    df = pd.read_csv('/Users/eliecochard/Desktop/XHEC 2/Artefact/xhec-mlops-project-student/abalone.csv')

    # Compute target
    df['Age'] = df['Rings'] + 1.5

    # Encode categorical columns
    df = pd.get_dummies(df, columns=['Sex'], drop_first=True)

    # Extract X and y
    X, y = df.drop(["Age", "Rings"], axis=1), df["Age"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

    # StandardScaler
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train model
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    model.fit(X_train_scaled, y_train)

    # Evaluate model
    y_pred_train = model.predict(X_train_scaled)
    compute_metrics(y_train, y_pred_train)  

    # Evaluate model on test set
    y_pred = model.predict(X_test_scaled)
    compute_metrics(y_test, y_pred)

    # Log your model
    mlflow.sklearn.log_model(model, "linear_regression_model")